# 0. 코랩 <-> 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers accelerate
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 39.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import textwrap
from peft import PeftModel

# 1. 지정한 경로 가져옴
model_path = "/content/drive/MyDrive/DILAB/llama3-Korean-Bllossom-8B"

# 2. 지정한 경로에서 토크나이저, Model들을 불러온다
  # 학습할 때 진행했던 토크나이저, 양자화, LoRA 설정 똑같이 설정
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
# PAD 토큰이 없다면, EOS 토큰으로 해당 역할 대체
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

# 3. 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map = "auto", # GPU 자동 할당
    torch_dtype = "auto" # fp16 등 자동 감지
)


# 4. K/V 캐시(past_key_values) 사용 설정
  # 학습 중에는 gradient checkpointing과 충돌/경고가 나거나, 캐시 유지가 메모리를 더 먹을 수 있음
  # 따라서, 보통 학습할 때는 K/V 옵션을 끄며, 추론&생성 시에는 킨다
model.config.use_cache = True

# 5. **임베딩 크기 일치화**: 토크나이저 길이에 맞춰 리사이즈(← 이 것이 핵심!)
model.resize_token_embeddings(len(tokenizer))


# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",     # GPU 할당
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Device set to use cuda:0


In [ ]:

def generate_response(user_input, history, temperature=0.7, top_p=0.9, max_tokens=500):
  global pipe
  global his
  his = history
  print(history)
  messages = [{"role":"assistant", "content":"너는 뛰어난 AI 어시스턴트야. 너는 상대방과 대화를 하는 챗봇이야."}]

  for h in history:
    messages.append({"role":"user", "content": h[0]})
    messages.append({"role":"assistant", "content": h[1]})

  messages.append({"role":"user", "content": user_input})

  prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

  # 텍스트 생성
  output = pipe(
      prompt,
      max_new_tokens=max_tokens,     # 생성할 최대 토큰 수
      do_sample=True,        # 샘플링 방식 사용 (더 자연스럽게)
      temperature=temperature,       # 창의성 조절 (낮을수록 보수적)
      top_p=top_p,              # nucleus sampling
      eos_token_id=tokenizer.eos_token_id,  # 문장이 끝났음을 뜻하는 eos_token_id를 명시하여, 모델이 끝을 알게 하기
      return_full_text=False
  )
  # 출력 결과 확인
  output_text = output[0]["generated_text"]

  #wrapped_text = textwrap.fill(output_text, width=70)
  return output_text


In [ ]:
import gradio as gr

his = [] # 대화 기록 저장용 리스트

iface = gr.ChatInterface(
    fn=generate_response,
    title="Llama3 ChatBot",
    description = "Let's talking with me"
)
iface.launch(share = True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://73102f25dde8afa470.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
